In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from thesislib.utils import pathutils
import json

In [3]:
nlice_data_csv = pathutils.get_data_file('05_27_nlice/nlice/data/parsed/train.csv_sparse.csv')
symptom_map_file = pathutils.get_data_file("05_27_nlice/basic/symptom_db.json")

In [4]:
import pandas as pd
import numpy as np

In [5]:
with open(symptom_map_file) as fp:
    symptom_db = json.load(fp)

num_symptoms = len(symptom_db)

In [6]:
df = pd.read_csv(nlice_data_csv, index_col='Index')

In [7]:
from thesislib.utils.ml import models
from thesislib.utils import stringutils

In [8]:
sparsifier = models.ThesisAIMEDSymptomSparseMaker(num_symptoms=num_symptoms)

In [9]:
labels = df.LABEL.values
ordered_keys = ['GENDER', 'RACE', 'AGE', 'SYMPTOMS']
df = df[ordered_keys]

In [10]:
data = sparsifier.fit_transform(df)

In [11]:
data = data.asfptype()

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

In [13]:
split_t = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

In [14]:
train_data = None
train_labels = None
test_data = None
test_labels = None
for train_index, test_index in split_t.split(data, labels):
    train_data = data[train_index]
    train_labels = labels[train_index]
    test_data = data[test_index]
    test_labels = labels[test_index]

In [15]:
symptoms = [stringutils.slugify(symptom_db[item]) for item in sorted(symptom_db.keys())]

In [16]:
from lightgbm import Dataset, LGBMModel
import lightgbm as lgb

In [17]:
feature_names = ['gender', 'race', 'age'] + symptoms

In [18]:
categorical_data = ['gender', 'race', 'headache', 'limb-weakness', 'abdominal-pain', 'fever']

In [56]:
lgbm_train = Dataset(train_data, label=train_labels, feature_name=feature_names, categorical_feature=categorical_data)

In [20]:
lgbm_test = Dataset(
    test_data,
    label=test_labels,
    feature_name=feature_names,
    categorical_feature=categorical_data,
    reference=lgbm_train
)

In [21]:
num_labels = len(np.unique(labels))

In [52]:
parameters = {
    'application': 'multiclass',
    'num_class': num_labels,
    'tree_learner': 'voting',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

In [31]:
test_pred = bst.predict(test_data)

In [40]:
test_pred_labels = np.argmax(test_pred, axis=1)

In [43]:
test

(365, 8)